# Visual Covariance Test

$$
% latex macros.  this does not work when converting to latex.
\newcommand{\bm}[1]{\boldsymbol{#1}}
\DeclareMathOperator{\cov}{cov}
\DeclareMathOperator{\diag}{diag}
\newcommand{\ud}{\mathrm{d}}
\newcommand{\XX}{{\bm X}}
\newcommand{\VV}{{\bm V}}
\newcommand{\N}{\mathcal{N}}
\newcommand{\mmu}{{\bm \mu}}
\newcommand{\SSi}{{\bm \Sigma}}
\newcommand{\ssi}{{\bm \sigma}}
$$

Pick a covariance type. Then for $x_t$ and $v_t = \ud x_t/\ud t$, visually check that
$$
\begin{aligned}
\cov(x_t, v_s) & = \int_0^t \cov(v_u, v_s) \, \ud u \\
\cov(x_t, x_s) & = \int_0^t\int_0^s \cov(v_u, v_w) \, \ud u\, \ud w
\end{aligned}
$$
by comparing analytic formula to numerical integration with `scipy.integrate.quad` and `scipy.integrate.dblquad`.

In [1]:
import os
os.chdir('C:\\Users\\mohan\\Documents\\probDE')

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import uniform as runif
from scipy import integrate
import BayesODE as bo
from Tests.ode_bayes_star import ode_bayes_star

In [ ]:
npts = 50
# pick covariance type
type = "ex"
if type == "se":
    cov_vv = bo.cov_vv_se
    cov_xv = bo.cov_xv_se
    cov_xx = bo.cov_xx_se
elif type == "ex":
    cov_vv = bo.cov_vv_ex
    cov_xv = bo.cov_xv_ex
    cov_xx = bo.cov_xx_ex
elif type == "re":
    cov_vv = bo.cov_vv_re
    cov_xv = bo.cov_xv_re
    cov_xx = bo.cov_xx_re
else:
    raise ValueError("Invalid covariance type.")
# plot cov_vv
tmax = runif(low = 0, high = 10, size = 1)
gamma = runif(0, tmax/10, 1)
tseq = np.linspace(start = 0, stop = tmax, num = npts)
av = np.zeros((npts, 1))
for ii in range(0,npts):
    av[ii] = cov_vv(0, tseq[ii], gamma)
plt.rcParams['figure.figsize'] = [10, 6]
plt.subplot(3,2,(1,2))
plt.plot(tseq,av)
# integrate once and compare to analytic solution
t = runif(0, tmax, 1) # first time point
s = runif(0, tmax, 1) # second time point
tseq = np.linspace(start = 0, stop = tmax, num = npts)
xv_an = np.zeros((npts,1)) # analytic solution
xv_nu = xv_an # numeric solution
for ii in range(0,npts):
    xv_an[ii] = cov_xv(tseq[ii], s, gamma)
    xv_nu[ii],err = integrate.quad(cov_vv, 0, tseq[ii], args=(s,gamma))
plt.subplot(3,2,3)
plt.plot(tseq,xv_an,'r',tseq,xv_nu,'b')
plt.subplot(3,2,4)
plt.plot(tseq,abs(xv_an-xv_nu))
# integrate twice and compare to analytic solution
xx_an = np.zeros((npts,1)) # analytic solution
xx_nu = xx_an # numeric solution
for ii in range(0,npts):
    xx_an[ii] = cov_xx(tseq[ii], s, gamma)
    xx_nu[ii],err = integrate.dblquad(cov_vv, 0, tseq[ii], 0, s, args = (gamma))
plt.subplot(3,2,5)
plt.plot(tseq,xx_an,'r',tseq,xx_nu,'b')
plt.subplot(3,2,6)
plt.plot(tseq,abs(xx_an-xx_nu))

# Updating Algorithm Test

Let $T = (t_1, \ldots, t_N)$ denote the evaluation time points, and $\XX$ and $\VV$ denote vectors of length $N$ corresponding to $x(T)$ and $v(T)$.  The solution prior is then expressed as
$$
\begin{bmatrix} \XX \\ \VV \end{bmatrix} \sim \N\left(\begin{bmatrix} \mmu_X \\ \mmu_V \end{bmatrix}, \begin{bmatrix} \SSi_{XX} & \SSi_{XV} \\ \SSi_{VX} & \SSi_{VV} \end{bmatrix} \right).
$$
Now suppose that the $N$ model interrogations $\VV_\star$ and their predictive variances $\ssi_\star^2$ are given in advance.  Then joint distribution of $(\XX, \VV, \VV_\star)$ is
$$
\begin{bmatrix} \XX \\ \VV \\ \VV_\star \end{bmatrix} \sim \N\left(\begin{bmatrix} \mmu_X \\ \mmu_V \\ \mmu_V \end{bmatrix}, \begin{bmatrix} \SSi_{XX} & \SSi_{XV} & \SSi_{XV} \\ \SSi_{VX} & \SSi_{VV} & \SSi_{VV} \\ \SSi_{VX} & \SSi_{VV} & \SSi_{VV} + \diag(\ssi_\star^2) \end{bmatrix} \right),
$$
from which we can deduce calculate the mean and variance of the final update (normal) distribution, $p(\XX \mid \VV_\star)$.  This suggests the following unit test for the ODE solver:

1. Modify `ode_bayes` such that it accepts an optional input of $\VV_\star$ and $\ssi_\star^2$ (please use argument names consistent with the rest of the function).
2. Write a function to compute the mean and variance of $p(\XX \mid \VV_\star)$ based on $(\mmu_X, \mmu_V, \SSi_{XX}, \SSi_{XV}, \SSi_{VV}, \VV_\star)$.
3. Run `ode_bayes` with predetermined $(\VV_\star, \ssi_\star^2)$, and check that the mean and variance output is identical to that of step 2.

In [308]:
#Toy example
def f(x,t):
    return  3*(t+1/4) - x/(t+1/4)

a = 0
b = 4.75
x0 = 0
N = 100
tseq = np.linspace(a, b, N)
w = gamma = 1.0642211055276383
alpha = N/10

mu_v = np.zeros((N)) # prior mean of v(tseq)
mu_x = x0 + mu_v # prior mean of x(tseq)
Sigma_vv, Sigma_xx, Sigma_xv = bo.cov_prior(tseq, 'exp', gamma, alpha)

In [39]:
x = integrate.odeint(f,x0,tseq)
x = np.ndarray.flatten(x)
v = np.array([f(xl,tseq[i]) for i, xl in enumerate(x)])
sigma_star_avg = (max(v) - min(v))*0.05

upper = 3*sigma_star_avg/2
lower = sigma_star_avg/2

sigma_star = runif(lower, upper)
vstar = v+sigma_star*np.random.normal(0,1)

In [9]:
mu_star, Sigma_star = ode_bayes_star(mu_x, mu_v, Sigma_vv, Sigma_xx, Sigma_xv, vstar)

In [11]:
a, mu_x, Sigma_xx = bo.ode_bayes(f, tseq, x0, Sigma_vv, Sigma_xx, Sigma_xv, vstar)

C:\Users\mohan\Documents\probDE\BayesODE\ode_bayes.py:49: RuntimeWarning: covariance is not positive-semidefinite.
  return np.random.multivariate_normal(mu_x, Sigma_xx), mu_x, Sigma_xx


In [12]:
np.allclose(mu_star, mu_x), np.allclose(Sigma_star, Sigma_xx)

(True, True)

In [14]:
from pykalman import KalmanFilter

In [323]:
tseq2 = np.linspace(a,b,50)
def mvCond(tseq, x0):
    N = len(tseq)
    mu = [x0, 0]*N
    Sigma = np.matrix([[0]*N*2]*N*2)
    for i in range(N):
        for j in range(N):
            Sigma[2*i, 2*j] = bo.cov_xx_ex(tseq[i], tseq[j], gamma)
            Sigma[2*i, 2*j+1] = bo.cov_xv_ex(tseq[i], tseq[j], gamma)
            Sigma[2*i+1, 2*j] = bo.cov_xv_ex(tseq[j], tseq[i], gamma)
            Sigma[2*i+1, 2*j+1] = bo.cov_vv_ex(tseq[i], tseq[j], gamma)
    Sigma_00 = Sigma[0:2*N-2, 0:2*N-2]
    Sigma_10 = Sigma[2*N-2:, 0:2*N-2]
    Sigma_01 = Sigma[0:2*N-2, 2*N-2:]
    Sigma_11 = Sigma[2*N-2:, 2*N-2:]
    A = Sigma_10*np.linalg.pinv(Sigma_00)
    b = mu[2*N-2:]
    V = Sigma_11 - Sigma_10*np.linalg.pinv(Sigma_00)*Sigma_01
    return A,b,V
A, b, V = mvCond(tseq2, x0)
A

matrix([[ 0.12924721, -0.83928181, -0.51606141, -0.21277669, -1.24484165,
          0.43675397,  1.12166664, -0.41891638,  0.45795394, -0.04059249,
          1.52986899,  0.0470034 , -0.42179724,  0.0470034 , -1.80060441,
          0.14148325,  0.25160007,  0.0470034 ,  1.07184098,  0.0470034 ,
          0.48181025,  0.0470034 , -0.18487389, -0.04059249, -0.70036264,
         -0.04059249, -0.26373112, -0.41891638, -0.09447985, -0.41891638,
         -0.0875959 , -0.98945719, -0.37832388,  0.43675397, -0.57054081,
          0.43675397,  1.42621116,  0.9041046 ,  0.46735063,  1.75249475,
          0.84839015,  0.02813856, -1.72435619, -0.21277669, -0.24091525,
         -0.21277669,  0.76406325,  0.55128655, -0.08173305,  0.4695535 ,
          0.27123631,  0.74078981, -0.94633085, -0.20554104, -0.12706949,
         -0.33261053, -1.53465414, -1.86726467,  1.05823308, -0.80903159,
         -0.03025022, -0.83928181,  0.87097836, -0.83928181,  1.20402582,
          0.03169655, -0.26528297,  1.

In [313]:
Sigma_vv_00 = bo.cov_vv_ex(tseq[0], tseq[0], gamma)
Sigma_xx_00 = bo.cov_xx_ex(tseq[0], tseq[0], gamma)
Sigma_xv_00 = bo.cov_xv_ex(tseq[0], tseq[0], gamma)
Sigma_vv_10 = bo.cov_vv_ex(tseq[1], tseq[0], gamma)
Sigma_xx_10 = bo.cov_xx_ex(tseq[1], tseq[0], gamma)
Sigma_xv_10 = bo.cov_xv_ex(tseq[1], tseq[0], gamma)

Sigma_00 = np.matrix([[Sigma_xx_00, Sigma_xv_00],[Sigma_xv_00, Sigma_vv_00]])
Sigma_10 = np.matrix([[Sigma_xx_10, Sigma_xv_10],[Sigma_xv_10, Sigma_vv_10]])
A = Sigma_10*np.linalg.pinv(Sigma_00)

Sigma_vv_11 = bo.cov_vv_ex(tseq[1], tseq[1], gamma)
Sigma_xx_11 = bo.cov_xx_ex(tseq[1], tseq[1], gamma)
Sigma_xv_11 = bo.cov_xv_ex(tseq[1], tseq[1], gamma)
Sigma_vv_01 = bo.cov_vv_ex(tseq[0], tseq[1], gamma)
Sigma_xx_01 = bo.cov_xx_ex(tseq[0], tseq[1], gamma)
Sigma_xv_01 = bo.cov_xv_ex(tseq[0], tseq[1], gamma)

Sigma_11 = np.matrix([[Sigma_xx_11, Sigma_xv_11],[Sigma_xv_11, Sigma_vv_11]])
Sigma_01 = np.matrix([[Sigma_xx_01, Sigma_xv_01],[Sigma_xv_01, Sigma_vv_01]])
CC = Sigma_11 - Sigma_10*np.linalg.pinv(Sigma_00)*Sigma_01

In [322]:
def bayes_ode_kalman(fun, x0, tseq, gamma, alpha, A, CC):
    y0 = [x0, f(x0, tseq[0])]
    mu = [y0]*N
    Sigma = [[[0, 0], [0,0]]]*N
    yt = np.array([y0]*N)
    mu_nn = np.array((A*np.matrix(mu[0]).T)).T
    Sigma_nn = A*Sigma[0]*A.T + CC
    kf = KalmanFilter(initial_state_mean=mu[0], initial_state_covariance=Sigma[0], n_dim_obs=2) #Initialization of Kalman Filter
    for n in range(N-1):
        mu_nn = np.array((A*np.matrix(mu[n]).T)).T
        Sigma_nn = A*Sigma[n]*A.T + CC
        
        #Model Interrogation Step
        xt = np.random.normal(mu_nn[0][0], Sigma_nn[0,0])
        vt = fun(xt, tseq[n+1])
        yt[n+1] = [xt,vt]
        mu[n+1], Sigma[n+1] = kf.filter_update(mu_nn[0], Sigma_nn, yt[n+1])
    
    kf2 = KalmanFilter(initial_state_mean=mu[0], initial_state_covariance=Sigma[0], n_dim_obs=2)
    yt_mean = kf2.smooth(yt)[0]
    
    return mu, yt_mean

In [321]:
u,v = bayes_ode_kalman(f, x0, tseq, gamma, alpha, A, CC)


0.034043534881466865
0.03141856305434282
0.039936088889584255
0.04562077759709676
0.04704224223034151
0.05367545502556769
0.0649129180884571
0.0746666534934068
0.07288536644901575
0.08287960051610768
0.09503433441069745
0.09745828298849218
0.10164989201321054
0.11143369939430327
0.12101653254529453
0.1292578583483297
0.1274407972234591
0.1341685067358718
0.14278338264641244
0.15151069458372604
0.15016468533234878
0.1582710108535872
0.16570078814529832
0.17535398531174118
0.18374874444705094
0.18615526172613062
0.19066298417906472
0.19890485015057985
0.21189450493929302
0.21857543866112475
0.22337443284978925
0.22791909065740185
0.2343393176666263
0.2355211977063183
0.2454785302291117
0.24969650576320082
0.25910317109564834
0.2686765330946554
0.2752492103769758
0.28511205772699155
0.28899584204863726
0.2925324528163833
0.3026155596068269
0.3018254482566207
0.3178842543528953
0.31824911485435964
0.32920248558259274
0.33481359954093454
0.3401844485185477
0.3441713827476506
0.3540377937655

array([[  0.        ,   0.75      ],
       [  0.02146612,   0.82789743],
       [  0.03035483,   0.95400071],
       [  0.0381798 ,   1.08704159],
       [  0.04424849,   1.22668212],
       [  0.04894488,   1.37048049],
       [  0.05554391,   1.51108675],
       [  0.06401139,   1.64893438],
       [  0.07157735,   1.78894027],
       [  0.07605401,   1.93417207],
       [  0.0836993 ,   2.07501992],
       [  0.0921643 ,   2.21505887],
       [  0.09775927,   2.35901036],
       [  0.10365522,   2.50272236],
       [  0.11155651,   2.64428377],
       [  0.11958061,   2.78587537],
       [  0.12616879,   2.92904972],
       [  0.1296679 ,   3.07525617],
       [  0.13539412,   3.2193381 ],
       [  0.14234596,   3.36232687],
       [  0.14886036,   3.50571189],
       [  0.15272444,   3.6512782 ],
       [  0.15914826,   3.7948035 ],
       [  0.16644934,   3.93769448],
       [  0.17449896,   4.08009462],
       [  0.18169355,   4.22316136],
       [  0.18683296,   4.36767203],
 